In [269]:
import os
import openai
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType

# https://www.kaggle.com/datasets/tharunprabu/my-expenses-data

In [270]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [271]:
agent = create_csv_agent(ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0), 
                         '/Users/kamal/Documents/aisaver/data/expense_data_1_filtered.csv', 
                         verbose=True,
                         agent_type=AgentType.OPENAI_FUNCTIONS)

### Asking Questions

In [272]:
agent.run("Can you describe the dataset")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `df.describe()`


       Subcategory          INR  Note.1       Amount    Account.1
count          0.0    89.000000     0.0    89.000000    89.000000
mean           NaN   321.847640     NaN   309.423820   309.423820
std            NaN   998.681666     NaN   995.503226   995.503226
min            NaN    10.000000     NaN    10.000000    10.000000
25%            NaN    30.000000     NaN    30.000000    30.000000
50%            NaN    80.000000     NaN    80.000000    80.000000
75%            NaN   239.000000     NaN   200.000000   200.000000
max            NaN  8000.000000     NaN  8000.000000  8000.000000The `describe()` function provides a statistical summary of the numerical columns in the dataframe `df`. Here is the description of each column:

- `Subcategory`: This column has no non-null values, so there is no statistical summary available.
- `INR`: This column represents the amount in Indian Rupees (INR). The

'The `describe()` function provides a statistical summary of the numerical columns in the dataframe `df`. Here is the description of each column:\n\n- `Subcategory`: This column has no non-null values, so there is no statistical summary available.\n- `INR`: This column represents the amount in Indian Rupees (INR). The statistical summary shows the count, mean, standard deviation, minimum value, 25th percentile, 50th percentile (median), 75th percentile, and maximum value of the data.\n- `Note.1`: This column has no non-null values, so there is no statistical summary available.\n- `Amount`: This column represents the amount. The statistical summary shows the count, mean, standard deviation, minimum value, 25th percentile, 50th percentile (median), 75th percentile, and maximum value of the data.\n- `Account.1`: This column represents the account. The statistical summary shows the count, mean, standard deviation, minimum value, 25th percentile, 50th percentile (median), 75th percentile, a

In [273]:
agent.run("For each category, can you sum up the expenses for me?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `df.groupby('Category')['Amount'].sum()`


Category
Allowance           11000.00
Apparel              2590.00
Beauty                196.00
Food                 5335.00
Household            4800.00
Other                1790.00
Self-development      400.00
Social Life           513.72
Transportation        914.00
Name: Amount, dtype: float64Here is the sum of expenses for each category:

- Allowance: 11000.00
- Apparel: 2590.00
- Beauty: 196.00
- Food: 5335.00
- Household: 4800.00
- Other: 1790.00
- Self-development: 400.00
- Social Life: 513.72
- Transportation: 914.00

> Finished chain.


'Here is the sum of expenses for each category:\n\n- Allowance: 11000.00\n- Apparel: 2590.00\n- Beauty: 196.00\n- Food: 5335.00\n- Household: 4800.00\n- Other: 1790.00\n- Self-development: 400.00\n- Social Life: 513.72\n- Transportation: 914.00'

In [276]:
agent.run("""For each category, can you sum up the expenses for me?
          Once this has been done, can you suggest some ways to reduce expenses and increase savings?""")



> Entering new AgentExecutor chain...


OutputParserException: Could not parse tool input: {'name': 'python', 'arguments': "# Summing up expenses for each category\ncategory_expenses = df[df['Income/Expense'] == 'Expense'].groupby('Category')['Amount'].sum()\ncategory_expenses"} because the `arguments` is not valid JSON.

In [280]:
agent.run("""What expenses should I take a closer look at in order to reduce costs and increase savings. 
          Your response should have the following format: Expense1, Expense2, Expense3""")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `expenses = df[df['Income/Expense'] == 'Expense']
expenses_grouped = expenses.groupby('Category')['Amount'].sum()
top_expenses = expenses_grouped.nlargest(3)
top_expenses.index.tolist()`


['Food', 'Household', 'Apparel']You should take a closer look at the following expenses in order to reduce costs and increase savings: Food, Household, and Apparel.

> Finished chain.


'You should take a closer look at the following expenses in order to reduce costs and increase savings: Food, Household, and Apparel.'

### Ground truth data analysis

In [ ]:
import pandas as pd
df1=pd.read_csv('data/expense_data_1_filtered.csv')
df1.keys()

Index(['Date', 'Account', 'Category', 'Subcategory', 'Note', 'INR',
       'Income/Expense', 'Note.1', 'Amount', 'Currency', 'Account.1'],
      dtype='object')

In [ ]:
import pandas as pd
# import datetime.datetime as dt
from datetime import datetime

format_str = "%m/%d/%Y"

df=pd.read_csv('data/expense_data_1.csv')
# for each month sum up the expense per category

start_date = datetime.strptime('12/1/2021', format_str).date()
end_date = datetime.strptime('12/31/2021', format_str).date()

# df2 = pd.DataFrame(columns=['Category', 'Amount'])
dictinfo = {}
expenses = []

for index, row in df.iterrows():
    parsed_date = datetime.strptime(row['Date'].split(' ')[0], format_str).date()
    
    if start_date <= parsed_date <= end_date:
        # print(row['Date'], row['Category'], row['Amount'])
        if row['Income/Expense'] == 'Expense':
            expenses.append(row['Category'])

            dictinfo[row['Category']] = dictinfo.get(row['Category'], 0) + row['Amount']

# for k,v in dictinfo.items():
    # if k in expenses:
    #     dictinfo[k] = -v

In [ ]:
sorted(dictinfo.items(), key=lambda x: x[1])[::-1]

[('Food', 5335.0),
 ('Household', 4800.0),
 ('Apparel', 2590.0),
 ('Transportation', 914.0),
 ('Social Life', 513.72),
 ('Other', 480.0),
 ('Self-development', 400.0),
 ('Beauty', 196.0)]